# import dependencies

In [ ]:
!pip install shap

In [ ]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten


import shap



# data 

In [ ]:
data = pd.read_csv("../input/fer2013/fer2013.csv")

In [ ]:
data?

In [ ]:
data.head()

In [ ]:
data["Usage"][30000]

In [ ]:
data

In [ ]:
L=data.values.tolist()


In [ ]:
ls=[]
for i in L:
    if (i[0]==3 or i[0]==4):
        ls.append(i)

In [ ]:
len(ls)
ls[0]

In [ ]:
class_names = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise',
    6: 'neutral'
}

num_classes = len(class_names)
print(num_classes)



num_of_instances=len(ls)
print(num_of_instances)

In [ ]:
x_train, y_train, x_test, y_test = [], [], [], []
emotion, img, usage = ls[0][0],ls[0][1],ls[0][2]
val = img.split(" ")
            
pixels = np.array(val, 'float32')
        
emotion = keras.utils.to_categorical(emotion, num_classes)
if 'Training' in usage:
    y_train.append(emotion)
    x_train.append(pixels)
elif 'PublicTest' in usage:
    y_test.append(emotion)
    x_test.append(pixels)

x_train

In [ ]:
#original
x_train, y_train, x_test, y_test = [], [], [], []


for i in ls:
    try:
        emotion, img, usage = i[0],i[1],i[2]
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, num_classes)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("",end="")
print("done")

In [ ]:
y_train[0]

In [ ]:
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# construct a model

In [ ]:
#construct CNN structure
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
model.evaluate(x_test,y_test)

# explanability

In [ ]:
x_train = np.expand_dims(x_train, axis = 1)
x_test = np.expand_dims(x_test, axis = 1)

In [ ]:
x_train.shape

In [ ]:
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], x.copy()))
    return K.get_session().run(model.layers[layer].input, feed_dict)

In [ ]:
    sample=int(input())
    to_explain = x_train[[sample]]
    print(to_explain.shape)

In [ ]:
def explain(x_train, sample, layer):
    to_explain = x_train[[sample]]
    #print(to_explain.shape)
    
    e = shap.GradientExplainer(
        (model.layers[layer].input, model.layers[-1].output),
        map2layer(x_train, layer), 
        local_smoothing=0 # std dev of smoothing noise
    )
    
    shap_values,indexes = e.shap_values(map2layer(to_explain, layer), ranked_outputs=1)
    
    index_names = np.vectorize(lambda x: class_names[x])(indexes)
    
    shap.image_plot(shap_values, to_explain[0], index_names)

In [ ]:
sample = 100
for layer in range(5):
    print("layer ",layer,": ",model.layers[layer])
    explain(x_train, sample, layer)
    print("------------------")